In [1]:
from pathlib import Path
from scipy.sparse import coo_matrix, csr_matrix
from scipy.sparse.csgraph import shortest_path
from scipy.spatial.distance import cdist
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import trimesh
import ot
import ot.plot

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
import sys
sys.path.append('../code')
import FOT

In [3]:
TRAIN_PATH = Path('MPI-FAUST/training/registrations')

In [4]:
def load_ply_mesh(p):
    """
    p: Path to .ply
    return: (V, F) with V:(n,3) float64, F:(m,3) int
    """
    m = trimesh.load_mesh(str(p), process=False)
    # trimesh는 faces가 (m,3), vertices가 (n,3)
    V = np.asarray(m.vertices, dtype=np.float64)
    F = np.asarray(m.faces, dtype=np.int32)
    return V, F

# 사용 가능한 파일 확인
files = sorted(TRAIN_PATH.glob("*.ply"))
print(f"#files: {len(files)}")
if files[:3]: 
    for f in files[:3]:
        print("e.g.", f.name)

#files: 100
e.g. tr_reg_000.ply
e.g. tr_reg_001.ply
e.g. tr_reg_002.ply


In [5]:
# V, F = load_ply_mesh(files[0])
# print("V shape:", V.shape, "F shape:", F.shape)
# print("bbox min:", V.min(0), "bbox max:", V.max(0))
# print("mean edge length ~", np.linalg.norm(V[F[:,0]]-V[F[:,1]], axis=1).mean())

In [6]:
def to_mesh(V, F, colors=None):
    mesh = o3d.geometry.TriangleMesh(
        o3d.utility.Vector3dVector(V),
        o3d.utility.Vector3iVector(F)
    )
    mesh.compute_vertex_normals()
    if colors is not None:
        mesh.vertex_colors = o3d.utility.Vector3dVector(colors)
    return mesh

# mesh = to_mesh(V, F)
# o3d.visualization.draw_geometries([mesh])  # 새 창으로 3D 뷰어

In [7]:
V_s, F_s = load_ply_mesh(files[0])
V_t, F_t = load_ply_mesh(files[1])
n_s, n_t = len(V_s), len(V_t)

# mesh_s = to_mesh(V_s, F_s)
# mesh_t = to_mesh(V_t, F_t)

# # 한쪽을 옆으로 이동해서 나란히 띄우기
# offset = np.array([1.2*(V_s[:,0].ptp()), 0, 0])  # x축으로 bbox폭 만큼
# mesh_t.translate(offset)

# o3d.visualization.draw_geometries([mesh_s, mesh_t])

In [8]:
# def random_colormap(n):
#     cmap = plt.colormaps.get_cmap('hsv')  # 새 API
#     colors = cmap(np.linspace(0, 1, n))[:, :3]
#     return colors

# n_parts = 10  # 예: 신체 10영역
# labels = np.random.randint(0, n_parts, len(V_s))  # 예시: 랜덤 파트 라벨
# colors_src = random_colormap(n_parts)[labels]
colors_src = (V_s - V_s.min(0)) / (V_s.max(0) - V_s.min(0) + 1e-12)
mesh_src = to_mesh(V_s, F_s, colors_src)

In [9]:
# 랜덤 하드 매핑 (source마다 하나의 target)
# vertex 개수가 다르면 일부는 중복 허용 (작은 쪽 기준)
# idx_t = np.random.choice(n_t, size=n_s, replace=False)

# # OT matrix T: (n_s, n_t)
# T = np.zeros((n_s, n_t), dtype=np.float64)
# T[np.arange(n_s), idx_t] = 1.0

In [10]:
def transfer_colors_soft(T, colors_src): 
    w = T.sum(0, keepdims=True) + 1e-12 
    colors_t = (T.T @ colors_src) / w.T 
    return np.clip(colors_t, 0, 1) 

In [11]:
# colors_t = transfer_colors_soft(T, colors_src) 
# mesh_t = to_mesh(V_t, F_t, colors_t)

# offset = np.array([1.2*V_s[:, 0].ptp(), 0, 0])
# mesh_t.translate(offset)
# o3d.visualization.draw_geometries([mesh_src,mesh_t])

In [12]:
#### Naive OT ####

# M = ot.dist(V_s,V_t, metric='sqeuclidean')
# a = np.ones((n_s,)) / n_s
# b = np.ones((n_t,)) / n_t

# T_naive = ot.solve(M, a, b).plan

In [13]:
# colors_naive = transfer_colors_soft(T_naive, colors_src) 
# mesh_naive = to_mesh(V_t, F_t, colors_naive)

# offset = np.array([1.2*V_s[:, 0].ptp(), 0, 0])
# mesh_naive.translate(offset)
# o3d.visualization.draw_geometries([mesh_src,mesh_naive])

In [14]:
# def plot_transport_heatmap(T, title="Transport plan T (rows: X_i, cols: Y_j)"):
#     fig, ax = plt.subplots(figsize=(6,5))
#     im = ax.imshow(T, aspect='auto')  # 행: X_i, 열: Y_j
#     ax.set_xlabel('target index j')
#     ax.set_ylabel('source index i')
#     ax.set_title(title)
#     cbar = plt.colorbar(im, ax=ax)
#     cbar.set_label('mass T[i,j]')
#     plt.tight_layout()
#     plt.show()

# def plot_transport_heatmap(T, n_sub=200, seed=0, title="Transport plan (sampled)"):
#     np.random.seed(seed)
#     n_source, n_target = T.shape
#     idx_s = np.random.choice(n_source, n_sub, replace=False)
#     idx_t = np.random.choice(n_target, n_sub, replace=False)
#     T_sub = T[np.ix_(idx_s, idx_t)]
    
#     fig, ax = plt.subplots(figsize=(6,5))
#     im = ax.imshow(T_sub, aspect='auto')
#     ax.set_xlabel(f'target index j (sampled {n_sub})')
#     ax.set_ylabel(f'source index i (sampled {n_sub})')
#     ax.set_title(title)
#     cbar = plt.colorbar(im, ax=ax)
#     cbar.set_label('mass T[i,j]')
#     plt.tight_layout()
#     plt.show()

In [15]:
# def gaussian_kernel_matrix(V, h):
#     """
#     Gaussian kernel matrix based on actual vertex coordinates.

#     Parameters
#     ----------
#     V : (n, d) array
#         Vertex coordinates.
#     h : float
#         Bandwidth parameter.

#     Returns
#     -------
#     Kh : (n, n) array
#         Symmetric kernel matrix with entries exp(-||v_i - v_j||^2 / (2h^2))
#     """
#     D2 = cdist(V, V, metric='sqeuclidean')  # ||V_i - V_j||^2
#     Kh = np.exp(- D2 / (2.0 * h**2))
#     np.fill_diagonal(Kh, 0.0)                # 자기쌍 제외
#     Kh = 0.5 * (Kh + Kh.T)                   # 수치적 대칭화
#     return np.sqrt(Kh)

# M = ot.dist(V_s,V_t, metric='sqeuclidean')
# C1 = ot.dist(V_s,V_s, metric='euclidean')
# C2 = ot.dist(V_t,V_t, metric='euclidean')

# a = np.ones((n_s,)) / n_s
# b = np.ones((n_t,)) / n_t

# avg_dist_X = np.mean(cdist(V_s, V_s))
# avg_dist_Y = np.mean(cdist(V_t, V_t))
# hX = 0.05 * avg_dist_X
# hY = 0.05 * avg_dist_Y


# KhX = gaussian_kernel_matrix(V_s, hX)
# KhY = gaussian_kernel_matrix(V_t, hY)
# eps = 1e-12
# C1p = KhX * C1
# C2p = KhY * C2
# C1p *= (C1.mean() + eps) / (C1p.mean() + eps)
# C2p *= (C2.mean() + eps) / (C2p.mean() + eps)

# alpha = 1.0

# T_kernel = ot.gromov.fused_gromov_wasserstein(M, C1p, C2p, a, b, alpha=alpha, loss_fun='square_loss')

In [16]:
# 1) 메쉬 그래프의 간선 길이(유클리드)로 가중치 부여
def build_edge_graph(V, F):
    """
    V: (n,3) vertex coords, F: (m,3) faces (int)
    return: CSR sparse adjacency with edge weights = edge lengths
    """
    n = V.shape[0]
    # 얼굴에서 간선 후보 추출
    E = np.vstack([
        F[:, [0, 1]],
        F[:, [1, 2]],
        F[:, [2, 0]]
    ])
    # 무방향 간선 정규화(작은 idx가 앞)
    E = np.sort(E, axis=1)
    # 중복 제거
    E = np.unique(E, axis=0)

    i, j = E[:, 0], E[:, 1]
    # 간선 길이 = ||v_i - v_j||
    L = np.linalg.norm(V[i] - V[j], axis=1)

    # 양방향으로 넣어 대칭 그래프 생성
    rows = np.concatenate([i, j])
    cols = np.concatenate([j, i])
    data = np.concatenate([L, L])

    W = coo_matrix((data, (rows, cols)), shape=(n, n)).tocsr()
    return W

# 2) 지오데식(모든 정점쌍 최단거리)
def geodesic_matrix(V, F):
    """
    return: D (n,n) geodesic distances on the mesh (float64)
    """
    W = build_edge_graph(V, F)
    # Dijkstra on undirected weighted graph
    D = shortest_path(W, directed=False, unweighted=False)  # (n,n)
    # 수치적 안전장치: 자기항 0, 대칭화
    np.fill_diagonal(D, 0.0)
    D = 0.5 * (D + D.T)
    return D

# # 3) 지오데식 기반 가우시안 커널
# def geodesic_kernel_matrix(V, F, h):
#     """
#     K_ij = exp( - d_geo(i,j)^2 / (2 h^2) ), diag=0, sqrt로 완화
#     """
#     D = geodesic_matrix(V, F)
#     Kh = np.exp(-(D**2) / (2.0 * (h**2)))
#     np.fill_diagonal(Kh, 0.0)
#     Kh = 0.5 * (Kh + Kh.T)
#     return np.sqrt(Kh)

# # ---------- 여기서부터 기존 파이프라인 치환 ----------
# # (V_s, F_s), (V_t, F_t) 는 FAUST source/target 메쉬
# n_s, n_t = V_s.shape[0], V_t.shape[0]
# a = np.ones(n_s) / n_s
# b = np.ones(n_t) / n_t

# # (A) 구조거리: 유클리드 대신 지오데식으로
Dx = geodesic_matrix(V_s, F_s).astype(np.float64)
Dy = geodesic_matrix(V_t, F_t).astype(np.float64)

# # (B) 커널 대역폭: 지오데식 스케일에 맞춰 추정(비대각 원소 평균/중앙값 기반)
# def offdiag_mean(D):
#     m = D.shape[0]
#     return D[~np.eye(m, dtype=bool)].mean()

# avg_geo_X = offdiag_mean(Dx)
# avg_geo_Y = offdiag_mean(Dy)
# hX = 10 * avg_geo_X   # 필요시 0.03~0.1 범위 탐색 추천
# # hY = 0.05 * avg_geo_Y

# # KhX = geodesic_kernel_matrix(V_s, F_s, hX)
# # KhX = 1
# # KhY = geodesic_kernel_matrix(V_t, F_t, hY)

# # (C) 구조항에 커널 가중을 곱해 local coherence 강화
# eps = 1e-12
# C1p = KhX * C1
# C2p = KhX * C2
# # 스케일 정렬(원래 C1, C2와 평균 스케일 맞추기) → FGW 수치 안정성↑
# C1p *= (C1.mean() + eps) / (C1p.mean() + eps)
# C2p *= (C2.mean() + eps) / (C2p.mean() + eps)

# # (D) 특성거리 M: 두 그래프 간의 point feature 거리
# # - 가장 권장: HKS/WKS/SHOT 등 불변 특징의 L2거리
# # - 대안(빠른 실험): 좌표 유클리드(정렬/정규화 후) 또는 색/법선 특징
# # 예) 좌표유클리드(현 상태 유지):
# M = cdist(V_s, V_t, metric='sqeuclidean')

In [ ]:
model = FOT.ConvexFusedTransport(
    alpha=0.8,
    h=1,
    kappa='decreasing_exp',
    metric=None,
    fw_max_iter=200,
    fw_stepsize='classic',
    tol=1e-7,
    lmo_method='emd',
    pre_DX = Dx,
    pre_DY = Dy
).fit(X=V_s,Y=V_t,FX=V_s,FY=V_t, return_hard_assignment=True)

C:\Users\junhy\AppData\Local\Programs\Python\Python312\Lib\site-packages\ot\lp\_network_simplex.py:332: UserWarning: numItermax reached before optimality. Try to increase numItermax.
  result_code_string = check_result(result_code)


In [ ]:
colors_FOT = transfer_colors_soft(model.P_, colors_src) 
mesh_FOT = to_mesh(V_t, F_t, colors_kernel)

offset = np.array([1.2*V_s[:, 0].ptp(), 0, 0])
mesh_FOT.translate(offset)

o3d.visualization.draw_geometries([mesh_src,mesh_FOT])

In [60]:
# alpha = 0.7  # pure GW라면 1.0, 특징도 섞으려면 (0<alpha<1)로
# T_kernel = ot.gromov.fused_gromov_wasserstein(
#     M, C1p, C2p, a, b,
#     alpha=alpha, loss_fun='square_loss'
# )

# alpha = 0.2  # pure GW라면 1.0, 특징도 섞으려면 (0<alpha<1)로
# T_kernel_a = ot.gromov.fused_gromov_wasserstein(
#     M, C1p, C2p, a, b,
#     alpha=alpha, loss_fun='square_loss'
# )

# alpha = 0.7  # pure GW라면 1.0, 특징도 섞으려면 (0<alpha<1)로
# T_kernel_b = ot.gromov.fused_gromov_wasserstein(
#     M, C1p, C2p, a, b,
#     alpha=alpha, loss_fun='square_loss'
# )

# alpha = 0.9  # pure GW라면 1.0, 특징도 섞으려면 (0<alpha<1)로
# T_kernel_c = ot.gromov.fused_gromov_wasserstein(
#     M, C1p, C2p, a, b,
#     alpha=alpha, loss_fun='square_loss'
# )

# alpha = 0.9  # pure GW라면 1.0, 특징도 섞으려면 (0<alpha<1)로
# T_kernel_d = ot.gromov.fused_gromov_wasserstein(
#     M, C1p, C2p, a, b,
#     alpha=alpha, loss_fun='square_loss'
# )

In [61]:
# colors_naive = transfer_colors_soft(T_naive, colors_src) 
# mesh_naive = to_mesh(V_t, F_t, colors_naive)

# colors_kernel = transfer_colors_soft(T_kernel, colors_src) 
# mesh_kernel = to_mesh(V_t, F_t, colors_kernel)

# offset = np.array([1.2*V_s[:, 0].ptp(), 0, 0])
# mesh_naive.translate(offset)

# offset = np.array([2.4*V_s[:, 0].ptp(), 0, 0])
# mesh_kernel.translate(offset)

# o3d.visualization.draw_geometries([mesh_src,mesh_naive,mesh_kernel])

In [ ]:
colors_naive = transfer_colors_soft(T_naive, colors_src) 
mesh_naive = to_mesh(V_t, F_t, colors_naive)

colors_kernel_a = transfer_colors_soft(T_kernel_a, colors_src) 
mesh_kernel_a = to_mesh(V_t, F_t, colors_kernel_a)

colors_kernel_b = transfer_colors_soft(T_kernel_b, colors_src) 
mesh_kernel_b = to_mesh(V_t, F_t, colors_kernel_b)

colors_kernel_c = transfer_colors_soft(T_kernel_c, colors_src) 
mesh_kernel_c = to_mesh(V_t, F_t, colors_kernel_c)

colors_kernel_d = transfer_colors_soft(T_kernel_d, colors_src) 
mesh_kernel_d = to_mesh(V_t, F_t, colors_kernel_d)

offset = np.array([1.2*V_s[:, 0].ptp(), 0, 0])
mesh_naive.translate(offset)

offset = np.array([2.4*V_s[:, 0].ptp(), 0, 0])
mesh_kernel_a.translate(offset)

offset = np.array([3.6*V_s[:, 0].ptp(), 0, 0])
mesh_kernel_b.translate(offset)

offset = np.array([4.8*V_s[:, 0].ptp(), 0, 0])
mesh_kernel_c.translate(offset)

offset = np.array([6.0*V_s[:, 0].ptp(), 0, 0])
mesh_kernel_d.translate(offset)

o3d.visualization.draw_geometries([mesh_src,mesh_naive,mesh_kernel_a,mesh_kernel_b,mesh_kernel_c,mesh_kernel_d])

In [58]:
np.mean(M)

0.5145512784055482

In [46]:
np.mean(C1p)

0.9225204811940164

In [47]:
np.mean(C2p)

0.9043404089432612